In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np

from keras import initializers
from keras.initializers import RandomNormal
from keras.layers import (BatchNormalization, Conv2D, Conv2DTranspose, Dense,
                          Dropout, Flatten, Input, Reshape, UpSampling2D,
                          ZeroPadding2D)
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model, Sequential
from keras.optimizers import Adam

%matplotlib inline

In [2]:
def make_lines(array):
#     line_start=5
#     line_end=18
#     array_lines=np.copy(array)
#     for line_index in range(line_start, line_end):
#         array_lines[line_index] = np.full_like(array[line_index], np.nan)
#     return array_lines
    specificLinesIndexes = np.array([2,5,6,9,10,11,14,17,19])
    arrayLines=np.copy(array)
    for index in specificLinesIndexes:
        arrayLines[index] = np.full_like(array[index], np.nan)
    return arrayLines

In [3]:
numberOfFiles=0
try: 
    filesXtrain = os.listdir('../TrainingDataset/x_train/')
    numberOfFiles = len(filesXtrain)
except: print('File not found')
numberOfFiles-=11
testingSetSize = 100
mx_train_lines = np.empty((numberOfFiles-testingSetSize,24,144))
mx_train = np.empty((numberOfFiles-testingSetSize,24,144))
mx_train_lines_nan = np.empty((numberOfFiles-testingSetSize,24,144))
mx_test_lines = np.empty((testingSetSize,24,144))
mx_test_lines_nan = np.empty((testingSetSize,24,144))
mx_test = np.empty((testingSetSize,24,144))

for i in range(0,numberOfFiles-testingSetSize):
    mx_train[i] = np.load('../TrainingDataset/x_train/Y2_36_60_{}.npy'.format(i))
    mx_train_lines[i] = make_lines(mx_train[i])
    mx_train_lines_nan[i] = make_lines(mx_train[i])
for i in range(numberOfFiles-testingSetSize,numberOfFiles):
    mx_test[i-(numberOfFiles-testingSetSize)-1] = np.load('../TrainingDataset/x_train/Y2_36_60_{}.npy'.format(i))
    mx_test_lines[i-(numberOfFiles-testingSetSize)-1] = make_lines(mx_test[i-(numberOfFiles-testingSetSize)-1])
    mx_test_lines_nan[i-(numberOfFiles-testingSetSize)-1] = make_lines(mx_test[i-(numberOfFiles-testingSetSize)-1])

mx_train_lines=np.nan_to_num(mx_train_lines)
mx_train=np.nan_to_num(mx_train)
mx_test_lines=np.nan_to_num(mx_test_lines)
mx_test=np.nan_to_num(mx_test)

In [89]:
# Consistent results
np.random.seed(1337)

# The dimension of z
noise_dim = 144*24

batch_size = 50
steps_per_epoch = np.int16(numberOfFiles/batch_size)
epochs = 5

save_path = '../GAN/DCGAN/images/'

img_rows, img_cols, channels = 144, 24, 1

optimizer = Adam(0.0002, 0.5)

In [6]:
# Create path for saving images
if save_path != None and not os.path.isdir(save_path):
    os.mkdir(save_path)

In [91]:
def create_generator():
    generator = Sequential()
    
    # Starting size
    d = 4
    generator.add(Dense(3*18*256, kernel_initializer=RandomNormal(0, 0.02), input_dim=noise_dim))
    generator.add(LeakyReLU(0.2))
    # 4x4x256
    generator.add(Reshape((3, 18, 256)))
    
    # 8x8x128
    generator.add(Conv2DTranspose(128, (4, 4), strides=2, padding='same', kernel_initializer=RandomNormal(0, 0.02)))
    generator.add(LeakyReLU(0.2))
    
    # 16x16*128
    generator.add(Conv2DTranspose(128, (4, 4), strides=2, padding='same', kernel_initializer=RandomNormal(0, 0.02)))
    generator.add(LeakyReLU(0.2))
    
    # 32x32x128
    generator.add(Conv2DTranspose(128, (4, 4), strides=2, padding='same', kernel_initializer=RandomNormal(0, 0.02)))
    generator.add(LeakyReLU(0.2))
    
    # 32x32x3
    generator.add(Conv2D(channels, (3, 3), padding='same', activation='tanh', kernel_initializer=RandomNormal(0, 0.02)))
    
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)
    return generator

In [61]:
def create_discriminator():
    discriminator = Sequential()
    
    discriminator.add(Conv2D(64, (3, 3), padding='same', kernel_initializer=RandomNormal(0, 0.02), input_shape=(img_cols, img_rows, channels)))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Conv2D(128, (3, 3), strides=2, padding='same', kernel_initializer=RandomNormal(0, 0.02)))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Conv2D(128, (3, 3), strides=2, padding='same', kernel_initializer=RandomNormal(0, 0.02)))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Conv2D(256, (3, 3), strides=2, padding='same', kernel_initializer=RandomNormal(0, 0.02)))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Flatten())
    discriminator.add(Dropout(0.4))
    discriminator.add(Dense(1, activation='sigmoid', input_shape=(img_cols, img_rows, channels)))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
    return discriminator

In [92]:
discriminator = create_discriminator()
generator = create_generator()

# Make the discriminator untrainable when we are training the generator.  This doesn't effect the discriminator by itself
discriminator.trainable = False

# Link the two models to create the GAN
gan_input = Input(shape=(noise_dim,))
fake_image = generator(gan_input)

gan_output = discriminator(fake_image)

gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=optimizer)

In [93]:
# Constant noise for viewing how the GAN progresses
static_noise = np.random.normal(0, 1, size=(10,noise_dim))

# Training loop
for epoch in range(20):
    for batch in range(steps_per_epoch):
        noise = np.random.normal(0, 1, size=(batch_size, noise_dim))
        real_x = mx_train[np.random.randint(0, mx_train.shape[0], size=batch_size)]
        real_x = np.expand_dims(real_x, 3)
        fake_x = generator.predict(noise)

        x = np.concatenate((real_x, fake_x))

        disc_y = np.zeros(x.shape[0])
        separator = np.int16(x.shape[0]/2)
        disc_y[:separator] = 0.9

        d_loss = discriminator.train_on_batch(x, disc_y)

        y_gen = np.ones(batch_size)
        g_loss = gan.train_on_batch(noise, y_gen)

    print(f'Epoch: {epoch} \t Discriminator Loss: {d_loss} \t\t Generator Loss: {g_loss}')
    np.save("../GAN/DCGAN/images/DCGAN_prediction_{}".format(epoch),generator.predict(static_noise).reshape(10,24,144))

Epoch: 0 	 Discriminator Loss: 0.45180705189704895 		 Generator Loss: 2.311074733734131
Epoch: 1 	 Discriminator Loss: 0.48293232917785645 		 Generator Loss: 1.5575053691864014
Epoch: 2 	 Discriminator Loss: 0.6606431603431702 		 Generator Loss: 1.0730115175247192
Epoch: 3 	 Discriminator Loss: 0.6875193119049072 		 Generator Loss: 0.9164425134658813
Epoch: 4 	 Discriminator Loss: 0.6992388963699341 		 Generator Loss: 1.155087947845459
Epoch: 5 	 Discriminator Loss: 0.6691053509712219 		 Generator Loss: 0.8490893840789795
Epoch: 6 	 Discriminator Loss: 0.6834601759910583 		 Generator Loss: 0.7793903350830078
Epoch: 7 	 Discriminator Loss: 0.7032247185707092 		 Generator Loss: 0.6595121622085571
Epoch: 8 	 Discriminator Loss: 0.7002910375595093 		 Generator Loss: 1.0083088874816895
Epoch: 9 	 Discriminator Loss: 0.6649432182312012 		 Generator Loss: 0.8457320332527161
Epoch: 10 	 Discriminator Loss: 0.6563476324081421 		 Generator Loss: 0.8764636516571045
Epoch: 11 	 Discriminator Loss: